### This is code from main.py

In [3]:
import os
import sys
import torch
import random
import argparse
import numpy as np
from GPT2.model import (GPT2LMHeadModel)
from GPT2.utils import load_weight
from GPT2.config import GPT2Config
from GPT2.sample import sample_sequence
from GPT2.encoder import get_encoder

In [5]:
filepath = '/Users/mohan/Desktop/cogworks/bwsi/ryan-sus/capstone/MadLib/gpt-2-Pytorch/gpt2-pytorch_model.bin'
state_dict = torch.load(filepath, map_location='cpu' if not torch.cuda.is_available() else None)

In [35]:
def loadModel(state_dict):
    seed = random.randint(0, 2147483647)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Model
    enc = get_encoder()
    config = GPT2Config()
    model = GPT2LMHeadModel(config)
    model = load_weight(model, state_dict)
    model.to(device)
    model.eval()
    
    return model, config, enc


In [42]:
def text_generator(model, config, enc, text,
    quiet = False,
    nsamples = 1,
    unconditional = True,
    batch_size = -1,
    length = -1,
    temperature = 0.7,
    top_k = 40):
     
    
    text_list = []
    
    if batch_size == -1:
        batch_size = 1
    assert nsamples % batch_size == 0
    

    if length == -1:
        length = config.n_ctx // 2
    elif length > config.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % config.n_ctx)
        
        
    context_tokens = enc.encode(text)
    
    
    generated = 0
    for _ in range(nsamples // batch_size):
        out = sample_sequence(
            model=model, length=length,
            context=context_tokens  if not unconditional else None,
            start_token=enc.encoder['<|endoftext|>'] if unconditional else None,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k, device="cpu"
        )
        out = out[:, len(context_tokens):].tolist()
        for i in range(batch_size):
            generated += 1
            text = enc.decode(out[i])
            text_list.append(text)
            
    return text_list[0]
            
            

In [47]:
model, config, enc = loadModel(state_dict)


In [63]:
text = text_generator(model, config, enc, "Marvel superheroes are the best. My favorite movie was Endgame.",
    quiet = False,
    nsamples = 1,
    unconditional = False,
    batch_size = -1,
    length = 50,
    temperature = 0.7,
    top_k = 40)

100%|███████████████████████████████████████████| 50/50 [00:07<00:00,  6.38it/s]


In [64]:
print(text)

 It's not a bad movie. It's a great movie. It's not a bad movie. And it's not a bad movie because it's so good.

The movie that really opened my eyes was Superman Returns. It's a huge
